In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

data=pd.read_csv("Creditcard_data.csv")

minor=data[data["Class"]==1]
major=data[data["Class"]==0].sample(n=len(minor),random_state=42)
balanced=pd.concat([major,minor]).sample(frac=1,random_state=42)

samplings=[]
samplings.append(balanced.sample(frac=0.8,random_state=42))
samplings.append(balanced.iloc[::2])
samplings.append(balanced.groupby("Class",group_keys=False).apply(lambda x:x.sample(frac=0.8,random_state=42)))
balanced["cluster"]=balanced.index%4
samplings.append(balanced[balanced["cluster"].isin([0,1])].drop("cluster",axis=1))
samplings.append(balanced.sample(frac=1,replace=True,random_state=42))

models=[
LogisticRegression(max_iter=1000),
DecisionTreeClassifier(),
RandomForestClassifier(),
KNeighborsClassifier(),
SVC()
]

results=pd.DataFrame(index=["M1","M2","M3","M4","M5"],columns=["Sampling1","Sampling2","Sampling3","Sampling4","Sampling5"])

for i,model in enumerate(models):
    for j,sample in enumerate(samplings):
        X=sample.drop("Class",axis=1)
        y=sample["Class"]
        Xtr,Xte,ytr,yte=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
        model.fit(Xtr,ytr)
        acc=accuracy_score(yte,model.predict(Xte))*100
        results.iloc[i,j]=round(acc,2)

print(results)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_3740\3408355239.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  samplings.append(balanced.groupby("Class",group_keys=False).apply(lambda x:x.sample(frac=0.8,random_state=42)))


   Sampling1 Sampling2 Sampling3 Sampling4 Sampling5
M1     66.67      50.0       0.0     100.0     100.0
M2     66.67      50.0     33.33     100.0     100.0
M3     66.67       0.0     33.33       0.0     100.0
M4     66.67      50.0     33.33      50.0      75.0
M5     66.67     100.0     33.33      50.0      75.0
